In [1]:
import numpy as np
from time import time
from PIL import Image
import torch
import torch.nn as nn
from IPython.display import clear_output
import matplotlib.pyplot as plt
from backbones import get_model
from privacy_pipeline import Pipeline
import torch.nn.functional as F
from tqdm.auto import tqdm

In [2]:
DEVICE = torch.device("cuda")

# 获取 & 载入模型
net = get_model('r100',fp16=False).eval().to(DEVICE)
net.load_state_dict(torch.load('/private/data/backbone.pth'))

#这里的batch_size是干啥的: 用来在恢复的时候 一次sample几个出来
batch_size = 80
image_dim = 112

def tensorToImage(im):return ((im+1)/2).permute(1, 2, 0).cpu().detach().numpy()

from dataset import MXFaceDataset,DataLoaderX
# 载入数据集
train_set = MXFaceDataset('/private/data/ms1m-retinaface-t1',local_rank=0)
train_loader = DataLoaderX(local_rank=0,dataset=train_set,batch_size=1)

In [3]:
%%time
# test dataset time
l = []
with torch.no_grad():
    test = DataLoaderX(local_rank = 0,dataset = train_set, batch_size = 200)
    for X,y in tqdm(test):
        for i in range(4):
            xx=X[y==(torch.ones(y.shape[0]).cuda())*i]
            l.append(xx)
        break

  0%|          | 0/25898 [00:00<?, ?it/s]

CPU times: user 24.4 s, sys: 97.7 ms, total: 24.5 s
Wall time: 1.56 s


In [4]:
!ls save

emb_average.pt


In [5]:
for X in l:
    xx = X.sum(dim=0,keepdim=True)/X.shape[0]
    print(xx.shape)
    emb=F.normalize(net(xx))
    emb_center=torch.load('save/emb_average.pt')
    print((emb**2).sum(),(emb_center**2).sum())
    print((emb*emb_center).sum())
    # plt.imshow(tensorToImage(xx[0]))
    # plt.show()
    break

torch.Size([1, 3, 112, 112])
tensor(1., device='cuda:0', grad_fn=<SumBackward0>) tensor(1.0000, device='cuda:0')
tensor(0.4912, device='cuda:0', grad_fn=<SumBackward0>)


In [6]:
with torch.no_grad():
    for i in train_loader:
        l = i[0]
        break
    emb_target = net(l)
    print(emb_target,torch.norm(emb_target))
# plt.imshow(tensorToImage(l[0]))

tensor([[ 1.2480e-01,  8.3153e-01, -7.7304e-01, -4.7051e-01,  7.2489e-01,
         -8.0591e-01, -3.7792e-01,  2.3763e-01, -7.6377e-01,  2.3465e+00,
         -9.9284e-01,  1.3887e-01, -6.0216e-01, -2.0728e+00, -1.3546e-01,
          8.4928e-01, -1.9278e+00,  1.2930e+00, -2.5405e+00,  6.9330e-01,
          1.4012e+00, -1.6907e+00,  3.1183e+00, -1.6562e+00,  1.0623e+00,
          4.3762e-02, -8.9675e-01, -2.8476e-01, -6.3807e-01, -3.4241e-01,
          2.1793e+00,  1.9215e-01, -9.7221e-01,  4.7018e-01, -1.0083e+00,
         -1.3206e+00, -1.3005e-02, -2.4216e-03,  3.4829e-01, -1.3502e-01,
          3.2057e-01,  8.9530e-01,  1.7754e+00,  2.3679e+00, -3.7300e-01,
          2.9850e+00,  2.8425e-01,  2.3323e-01,  1.7243e+00, -4.8577e-01,
         -1.1005e+00, -4.3982e-01,  1.3883e+00, -1.9627e-01, -5.0318e-01,
         -1.1315e+00, -1.1354e+00, -6.2007e-01, -2.9354e-01, -1.0758e+00,
         -1.9350e-01, -1.1304e+00, -9.5838e-01,  2.4308e+00,  3.8741e-01,
          7.9843e-01, -4.1580e-01,  4.

In [7]:
pipeline = Pipeline(emb_target,
                    net,
                    DEVICE,
                    dim=image_dim,
                    batch_size=batch_size,
                    sym_part=1,
                    multistart=True,
                    gauss_amplitude=0.02,
                    color=False,
                    )

In [8]:
(emb_target**2).sum()

tensor(621.8265, device='cuda:0')

In [9]:
cosines_target = []
facenet_sims = []

In [ ]:
with torch.no_grad():
    for i in range(15000):
        start = time()
        
        recovered_face, cos_target = pipeline()
        
        cosines_target.append(cos_target)
        
        
        if i % 10 == 0:
            clear_output(wait=True)
            recovered_face = np.transpose(recovered_face.cpu().detach().numpy(),(1,2,0))
            recovered_face = recovered_face - np.min(recovered_face)
            recovered_face = recovered_face / np.max(recovered_face)
            
            plt.figure(dpi=130)
            plt.imshow(recovered_face)
            plt.title(f"iterations {i*pipeline.batch_size}   cos_target={round(cos_target,3)}   norm={round(pipeline.norm,3)} best={(round(pipeline.best_value.item(),3))}")
            plt.show()
            
            plt.plot(cosines_target)
            plt.grid()
            plt.title("cos with a target embedding vs iters")
            plt.show()
# 最开始的一种, 恢复出来可以说非常像了吧